This is a notebook to visualize photobleaching in our experiments

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib qt

In [7]:
import glob
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from janelia_core.dataprocessing.baseline import percentile_filter_1d
from janelia_core.dataprocessing.dataset import ROIDataset

## Parameters go here

In [4]:
ps = dict()

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'

# Subfolder containing the dataset for each subject
ps['dataset_folder'] = 'extracted'

# Base folder where datasets are stored 
ps['dataset_base_folder'] =r'K:\\SV4'

# Key in each dataset with flouresence data
ps['fl_key'] = 'f_5_25_25'

# Key in each dataset with baseline data
ps['bl_key'] = 'bl_5_25_25'

## Read in excel file specifying location of datasets

In [5]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Calculate average $\Delta F / F$ in all ROIS across time for each dataset

In [37]:
n_datasets = 5 #len(data_locs)
vls = [None]*n_datasets
for d_i in range(n_datasets):
    # Read in the data for this subject
    
    data_main_folder = data_locs['Main folder'][d_i]
    data_sub_folder = data_locs['Subfolder'][d_i]
    
    dataset_path = (Path(ps['dataset_base_folder']) / data_main_folder / data_sub_folder / Path(ps['dataset_folder']) / '*.pkl')
    dataset_file = glob.glob(str(dataset_path))[0]
    
    with open(dataset_file, 'rb') as f:
        dataset = ROIDataset.from_dict(pickle.load(f))
        
    fl_ts = dataset.ts_data[ps['fl_key']]['ts']
    
    fl_vls = dataset.ts_data[ps['fl_key']]['vls'][:]
    fl_mean = np.mean(fl_vls, axis=1)
    
    bl_vls = dataset.ts_data[ps['bl_key']]['vls'][:]
    bl_mean = np.mean(bl_vls, axis=1)
    
    new_bl = percentile_filter_1d(data=fl_mean, window_length=1001, filter_start=-500, write_offset=500, p=.5)
    
    vls[d_i] = {'fl_mean': fl_mean, 'bl_mean': bl_mean, 'new_bl': new_bl}
    
    print('Done with dataset ' + str(d_i + 1) + ' of ' + str(n_datasets) + '.')


Done with dataset 1 of 5.
Done with dataset 2 of 5.
Done with dataset 3 of 5.
Done with dataset 4 of 5.
Done with dataset 5 of 5.


## View all time courses

In [36]:
plt.figure(figsize=(12, 3))
for s_i, s_vls in enumerate(vls):
    plt.subplot(1, 5, s_i+1)
    plt.plot(s_vls['fl_mean'])
    plt.plot(s_vls['bl_mean'])
    plt.plot(s_vls['new_bl'])